In [ ]:
from ast import If
import json
import csv
import os
import glob
import copy
import numpy as np
import playerData

jsonFiles = []
csvFiles = []
playersArray = []
numNiveles = 4
nivel = -1
tiempoInicioNivel = 0
muertesPorNivel = np.zeros(numNiveles) #esto mide la suma de todas las muertes hechas por todos los jugadores?¿
saltosPorNivel = np.zeros(numNiveles)  #esto mide la suma de todos los saltos hechos por todos los jugadores?¿
numArchivos = 0
zone = -1
zonasEspeciales = None
isPlayerInZone = 0 #flag para las zonas especiales

specialZoneTimestamp = 0

def initZonasEspeciales():
    global zonasEspeciales
    zonasEspeciales = [[] for i in range(numNiveles)]
    zonasEspeciales[0].append([[23, 3], [26, 0]])

def getCollision(x, y, rect):
                #rect->arribaizq->x
        return (x > rect[0][0] and
                #rect->abajodcha->x
                x < rect[1][0] and
                #rect->arribaizq->y
                y < rect[0][1] and
                #rect->abajodcha->y
                y > rect[1][1])

def ProcessJson(f):
    Lines = f.readlines()
    count = 0
    # For each line process one event
    events = []
    for line in Lines:
        count += 1
        events.append(json.loads(line))
    return events

def ProcessCSV(f):

    players = []

    ncol = len(next(f))
    start = 3

    for row in f:
        answers = []
        name = str(row[start - 1])
        name = "".join([ch for ch in name if ch.isdigit()])   # getONLY the numbers in the name
        
        for x in range(start, ncol):
            answers.append(row[x])
        player = name, answers
        players.append(player)
    
    return players



# def Clean(f):
#     session_start = []
#     for i in range(0, f.readlines, 1):
#         line = f.readline[i]   

def ReadFiles():
    #List of files in folder
    for filename in glob.glob('**/*.json'):
        with open(filename, mode="r", encoding="utf-8") as f: # open files in readonly mode
            fName = f.name
            fName = "".join([ch for ch in fName if ch.isdigit()])   # getONLY the numbers in the name
            file = str(fName), ProcessJson(f)
            jsonFiles.append(file)
            player = playerData.Player(numNiveles, zonasEspeciales)
            playersArray.append(player)
            global numArchivos
            numArchivos += 1
            
    for filename in glob.glob('*.csv'):
        with open(filename, mode="r", encoding="utf-8") as f:
            csvreader = csv.reader(f)
            file = str(f.name), ProcessCSV(csvreader)
            csvFiles.append(file)
            


In [ ]:
def SessionStart(e,player):
    ID = e["sesion"]
    player.setID(ID)
def SessionEnd(e,player):
    player.end()
def LevelStart(e,player):
    global nivel
    nivel = e["nivel"] -3 #Los niveles estan codificados como nivel 3, 4, 5 y 6, asi que les resto esto para que se guarden como 0,1,2, y 3.
    player.resetPath(nivel)

    #si no ha llegado al final del nivel, pero si lo ha empezado antes (p.ej: si se buggea y reinicia), borra los datos de ese nivel
    if (not player.hasLevelBeenCompleted(nivel)):
        player.resetLevelInfo(nivel)
        global tiempoInicioNivel
        tiempoInicioNivel = e["tiempo"]

def LevelEnd(e, player):
    player.lvlEnd(e["tiempo"] - tiempoInicioNivel)

def Pause(e, player):
    player.pause(e["tiempo"])

def Unpause(e, player):
    player.pause(e["tiempo"])

def Death(e, player):
    muertesPorNivel[nivel] += 1
    player.death(nivel)

def Collision(e, player):
    player.collision(e["tag"], e["id"])


def Jump(e, player):
    saltosPorNivel[nivel] += 1
    player.jump(nivel)
    pos = e["x"], e["y"]
    player.addToPath(pos, nivel)
    if (CheckSpecialZone(e, player)):
        player.jumpEspecial(nivel, zone)
            

def PlayerPosition(e, player):
    pos = e["x"], e["y"]
    player.addToPath(pos, nivel)
    CheckSpecialZone(e, player)



def CheckSpecialZone(e, player):
    b = False
    for zona in range(len(zonasEspeciales[nivel])):
        if (getCollision(e["x"], e["y"], zonasEspeciales[nivel][zona])):
            b = True
            global isPlayerInZone
            global specialZoneTimestamp
            if(isPlayerInZone == 0):
                global zone
                zone = zona
                #guardar el tiempo inical
                isPlayerInZone = 1
                print("Tiempo inicial: ", e["tiempo"])
                specialZoneTimestamp = e["tiempo"] #primer salto
        elif (isPlayerInZone == 1):
            #ha salido de la zona especial
            isPlayerInZone = 0
            print("Tiempo final: ", e["tiempo"])
            time = e["tiempo"] - specialZoneTimestamp
            player.tiempoEspecial(nivel,zona,time)
    return b


In [ ]:
def FormularioEnd(p):
        player = playersArray[p]
        player.formularioInforme()

def Age(a, player):
        player.setAge(a)

def GeneralExperience(a, player):
        player.setGeneralExperience(a)

def PlatformExperience(a, player):
        player.setPlatformExperience(a)

def Walljump(a, player):
        player.setWalljump(a)

def Gravity(a, player):
        player.setGravity(a)
        
def LevelDificulty(level, a, player):
        player.setLevelDificulty(level, a)
        
def PlatformExperience(a, player):
        player.setPlatformExperience(a)

def Walljump(a, player):
        player.setWalljump(a)

def Gravity(a, player):
        player.setGravity(a)
        
def LevelDificulty(level, a, player):
        player.setLevelDificulty(level, a)

In [ ]:
def ProcessEvent(e,actualPlayer):
     nombre = e["nombre"]
     player = playersArray[actualPlayer] #algunos eventos usaran al jugador y otros no

     if nombre == "LEVEL_START":
          LevelStart(e,player)

     elif nombre == "SESSION_START":
          SessionStart(e,player)
          
     elif nombre == "SESSION_END": #descomentar para leer informes individuales
          SessionEnd(e,player)

     #no tiene en cuenta los eventos de debajo si ya ha completado el nivel antes
     if (player.hasLevelBeenCompleted(nivel)): 
          return

     if nombre == "LEVEL_END":
          LevelEnd(e, player)


     elif nombre == "PAUSE":
          Pause(e, player)

     elif nombre == "UNPAUSE":
          Unpause(e, player)

     elif nombre == "DEATH":
          Death(e,player)

     elif nombre == "COLLISION":
          Collision(e, player)

     elif nombre == "JUMP":
          Jump(e,player)

     elif nombre == "PLAYER_POSITION":
          PlayerPosition(e,player)


     #print(nivel,":",nombre)




In [ ]:
def ProcessAnswer(a, actualAnswer, actualPlayer):
    player = playersArray[actualPlayer]

    if(actualAnswer == 0):
        Age(a, player)

    elif(actualAnswer == 1):
        GeneralExperience(a, player)
        
    elif(actualAnswer == 2):
        PlatformExperience(a, player)
        
    elif(actualAnswer == 3):
        Walljump(a, player)
    
    elif(actualAnswer == 4):
        Gravity(a, player)
    
    elif(actualAnswer >= 5):
        LevelDificulty(actualAnswer - 5, a, player)
        

In [ ]:

def ProcessFiles():
    initZonasEspeciales()
    ReadFiles()
    actualPlayer = -1 #iterador para añadir datos de jugador
    for f in jsonFiles:
        actualPlayer+=1
        print("Processing player", actualPlayer)
        for e in f[1]:
            ProcessEvent(e,actualPlayer)
    
    for f in csvFiles:
        for p in f[1]:
            index = 0
            found = False
            actualPlayer = 0

            for n in playersArray:
                if str(n.ID) == p[0]:
                    actualPlayer = index
                    found = True
                    break
                else: index += 1

            if found == True:
                print("Processing formulary", actualPlayer)
                actualAnswer= 0
                for a in p[1]:
                    ProcessAnswer(a, actualAnswer, actualPlayer)
                    actualAnswer += 1
                FormularioEnd(actualPlayer)
            else: print("Wrong ID: ", p[0], "\n---------------------------------------------")
                
            
            
    print("------------------------------------------")
    print("Media de saltos por jugador en cada nivel:",saltosPorNivel/np.size(playersArray))
    print("Media de muertes por jugador en cada nivel:",muertesPorNivel/np.size(playersArray))

ProcessFiles()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.misc as sci
import imageio
from PIL import Image

#METODOS PARAMETRIZADOS
#def grafico_lineas():
def grafico_barras(x, altura, title, xlabel, ylabel):
        plt.bar(x,altura)
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.show()

def grafico_barras_apiladas(x,altura, labels, xlabel, ylabel, title):
        plt.bar(x,altura[0], label = labels[0])
        y = copy.deepcopy(altura[0])

        for i in range(len(altura)-1):
                plt.bar(x, altura[i+1], label = labels[i+1], bottom = y)
                y += altura[i + 1]

        plt.ylabel(ylabel)
        plt.xlabel(xlabel)
        plt.title(title)
        plt.legend()
        plt.show()
def grafico_barras_apiladas_prop(x,altura, labels, xlabel, ylabel, title, proporcion):
        np.seterr(divide='ignore', invalid='ignore')
        
        proportional = np.nan_to_num(np.divide(altura[0], proporcion))
        plt.bar(x, proportional, label = labels[0])
        y = copy.deepcopy(proportional)

        for i in range(len(altura)-1):
                proportional = np.nan_to_num(np.divide(altura[i + 1], proporcion))
                plt.bar(x, proportional, label = labels[i+1], bottom = y)
                y += proportional

        plt.ylabel(ylabel)
        plt.xlabel(xlabel)
        plt.title(title)
        plt.legend()
        plt.show()

def numJugadoresPorGrupo():
        rangosEdad = np.zeros(6)
        for p in playersArray:
                age = p.age
                if age == '0-9':
                        rangosEdad[0]+=1
                elif age == '10-19':
                        rangosEdad[1]+=1
                elif age == '20-29':
                        rangosEdad[2]+=1
                elif age == '30-39':
                        rangosEdad[3]+=1
                elif age == '40-59':
                        rangosEdad[4]+=1
                elif age == '60+':
                        rangosEdad[5]+=1
        return rangosEdad

def numJugadoresPorDif():
        numDificultad = [] #4 Dificultades
        for i in range(4):
                numDificultad.append(np.zeros(numNiveles))

        for p in playersArray:
                for n in range(len(p.level_dificulty)):
                        dificulty = p.level_dificulty[n]
                        if dificulty == 'Muy fácil':
                                numDificultad[0][n]+=1
                        elif dificulty == 'Fácil':
                                numDificultad[1][n]+=1
                        elif dificulty == 'Difícil':
                                numDificultad[2][n]+=1
                        elif dificulty == 'Muy Difícil':
                                numDificultad[3][n]+=1
 
        return numDificultad

def numJugadoresPorExp():
        numExp = np.zeros(4)
        for p in playersArray:
                exp = p.generalExp
                if exp == '1':
                        numExp[0]+=1
                elif exp == '2':
                        numExp[1]+=1
                elif exp == '3':
                        numExp[2]+=1
                elif exp == '4':
                        numExp[3]+=1

        return numExp

def numJugadoresPorDifWJ():
        num = np.zeros(4)
        for p in playersArray:
                exp = p.walljump
                if exp == '1':
                        num[0]+=1
                elif exp == '2':
                        num[1]+=1
                elif exp == '3':
                        num[2]+=1
                elif exp == '4':
                        num[3]+=1

        return num

def proporcionNumJugadores(datos):
        poblacion = numJugadoresPorGrupo()
        
        for i in range(len(poblacion)):
                #print("muertes:", datos[i], " num jugadores:", poblacion[i])
                if(poblacion[i] != 0):
                        datos[i] /= poblacion[i]



#CREACION DE GRAFICOS
def grafico_rangosEdad():
        x = ['0-9', '10-19', '20-29', '30-39', '40-59', '60+']
        rangosEdad = numJugadoresPorGrupo()
        grafico_barras(x,rangosEdad,'Número de jugadores por rango de edad', '', 'Número de jugadores')

def grafico_paths(): #grafico que mete todos los paths en una unica imagen
        limits = [[[-5.6,34.3],[0,9.4]], #nivel 1 (xlimit, ylimit)
                [[-50,42.5],[0,20]],
                [[-2.8,39],[-20,20]],
                [[-24.5,25.7],[-8.5,8.8]]]
        global numNiveles
        for i in range(numNiveles):
                
                img = plt.imread("CapturaLv"+ str((i + 1))+".png")
                plt.imshow(img, extent=[limits[i][0][0], limits[i][0][1], limits[i][1][0], limits[i][1][1]])
                plt.xlim = limits[i][0]
                plt.ylim = limits[i][1]
                for p in playersArray:
                        x = [pos[0] for pos in p.paths[i]]
                        y = [pos[1] for pos in p.paths[i]]
                        title = 'Caminos en el nivel', i+1
                        plt.title(title)
                        plt.plot(x,y)
                plt.show()

def graficoMuertesPorNivel():
        x = ['Nivel 1', 'Nivel 2', 'Nivel 3', 'Nivel 4']
        grafico_barras(x,muertesPorNivel,'Número de muertes totales por nivel', '', 'Muertes')

def graficoMuertesVSEdad():
        niveles = ['Nivel 1', 'Nivel 2', 'Nivel 3', 'Nivel 4']

        rangosEdad = [] #6 rangos de edad
        for i in range(6):
                rangosEdad.append(np.zeros(numNiveles))

        labels = ['0-9', '10-19', '20-29', '30-39', '40-59', '60+']
        for p in playersArray:
                age = p.age
                if age == '0-9':
                        rangosEdad[0]+=p.nDeaths
                elif age == '10-19':
                        rangosEdad[1]+=p.nDeaths
                elif age == '20-29':
                        rangosEdad[2]+=p.nDeaths
                elif age == '30-39':
                        rangosEdad[3]+=p.nDeaths
                elif age == '40-59':
                        rangosEdad[4]+=p.nDeaths
                elif age == '60+':
                        rangosEdad[5]+=p.nDeaths

        proporcionNumJugadores(rangosEdad)

        grafico_barras_apiladas(niveles,rangosEdad,labels,'Niveles','Media de muertes','Media de muertes por rango de edad')
        #grafico_barras_apiladas_prop(niveles,rangosEdad,labels,'Niveles','Muertes','Numero de muertes por rango de edad (Relativo)',muertesPorNivel)
               
def graficoMuertesVSDificultad():
        niveles = ['Nivel 1', 'Nivel 2', 'Nivel 3', 'Nivel 4']

        dificultades = [] #4 Dificultades
        for i in range(4):
                dificultades.append(np.zeros(numNiveles))

        labels = ['Muy fácil', 'Fácil', 'Difícil', 'Muy Difícil']
        for p in playersArray:
                for n in range(len(p.level_dificulty)):
                        dificulty = p.level_dificulty[n]
                        if dificulty == 'Muy fácil':
                                dificultades[0][n]+=p.nDeaths[n]
                        elif dificulty == 'Fácil':
                                dificultades[1][n]+=p.nDeaths[n]
                        elif dificulty == 'Difícil':
                                dificultades[2][n]+=p.nDeaths[n]
                        elif dificulty == 'Muy Difícil':
                                dificultades[3][n]+=p.nDeaths[n]
        
        grafico_barras_apiladas(niveles,dificultades,labels,'Niveles','Muertes','Numero de muertes según dificultad percibida (Absoluto)')
        grafico_barras_apiladas_prop(niveles,dificultades,labels,'Niveles','Muertes','Numero de muertes según dificultad percibida (Relativo)',muertesPorNivel)

def graficoMuertesVSExpGeneral():
        niveles = ['Nivel 1', 'Nivel 2', 'Nivel 3', 'Nivel 4']

        nivelesExperiencia = [] #4 Niveles de experiencia
        for i in range(4):
                nivelesExperiencia.append(np.zeros(numNiveles))

        labels = ['Nada', 'Poca', 'Media', 'Mucha']
        for p in playersArray:
                exp = p.generalExp
                if exp == '1':
                        nivelesExperiencia[0]+=p.nDeaths
                elif exp == '2':
                        nivelesExperiencia[1]+=p.nDeaths
                elif exp == '3':
                        nivelesExperiencia[2]+=p.nDeaths
                elif exp == '4':
                        nivelesExperiencia[3]+=p.nDeaths

        grafico_barras_apiladas(niveles,nivelesExperiencia,labels,'Niveles','Muertes','Numero de muertes por experiencia en videojuegos (Absoluto)')
        grafico_barras_apiladas_prop(niveles,nivelesExperiencia,labels,'Niveles','Muertes','Numero de muertes por experiencia en videojuegos (Relativo)',muertesPorNivel)

def graficoMuertesVSExpPlataformas():
        niveles = ['Nivel 1', 'Nivel 2', 'Nivel 3', 'Nivel 4']

        nivelesExperiencia = [] #4 Niveles de experiencia
        for i in range(4):
                nivelesExperiencia.append(np.zeros(numNiveles))

        labels = ['Nada', 'Poca', 'Media', 'Mucha']
        for p in playersArray:
                exp = p.platformExp
                if exp == '1':
                        nivelesExperiencia[0]+=p.nDeaths
                elif exp == '2':
                        nivelesExperiencia[1]+=p.nDeaths
                elif exp == '3':
                        nivelesExperiencia[2]+=p.nDeaths
                elif exp == '4':
                        nivelesExperiencia[3]+=p.nDeaths

        grafico_barras_apiladas(niveles,nivelesExperiencia,labels,'Niveles','Muertes','Numero de muertes por experiencia en plataformas (Absoluto)')
        grafico_barras_apiladas_prop(niveles,nivelesExperiencia,labels,'Niveles','Muertes','Numero de muertes por experiencia en plataformas (Relativo)',muertesPorNivel)

def graficoMediaMuertesVsDificultad():
        niveles = ['Nivel 1', 'Nivel 2', 'Nivel 3', 'Nivel 4']

        dificultades = [] #4 Dificultades
        for i in range(4):
                dificultades.append(np.zeros(numNiveles))

        labels = ['Muy fácil', 'Fácil', 'Difícil', 'Muy Difícil']
        for p in playersArray:
                for n in range(len(p.level_dificulty)):
                        dificulty = p.level_dificulty[n]
                        if dificulty == 'Muy fácil':
                                dificultades[0][n]+=p.nDeaths[n]
                        elif dificulty == 'Fácil':
                                dificultades[1][n]+=p.nDeaths[n]
                        elif dificulty == 'Difícil':
                                dificultades[2][n]+=p.nDeaths[n]
                        elif dificulty == 'Muy Difícil':
                                dificultades[3][n]+=p.nDeaths[n]
        
        poblacion = numJugadoresPorDif()
        
        for i in range(len(poblacion)):
                #print("muertes:", datos[i], " num jugadores:", poblacion[i])
                for j in range(len(poblacion[i])):

                        if(poblacion[i][j] != 0):
                                dificultades[i][j] /= poblacion[i][j]
        grafico_barras_apiladas(niveles,dificultades,labels,'Niveles','Media de Muertes','Media de muertes según dificultad percibida')


def graficoMediaMuertesVSExp():
        niveles = ['Nivel 1', 'Nivel 2', 'Nivel 3', 'Nivel 4']

        nivelesExperiencia = [] #4 Niveles de experiencia
        for i in range(4):
                nivelesExperiencia.append(np.zeros(numNiveles))

        labels = ['Nada', 'Poca', 'Media', 'Mucha']
        for p in playersArray:
                exp = p.generalExp
                if exp == '1':
                        nivelesExperiencia[0]+=p.nDeaths
                elif exp == '2':
                        nivelesExperiencia[1]+=p.nDeaths
                elif exp == '3':
                        nivelesExperiencia[2]+=p.nDeaths
                elif exp == '4':
                        nivelesExperiencia[3]+=p.nDeaths

        poblacion = numJugadoresPorExp()
        
        for i in range(len(poblacion)):
                #print("muertes:", nivelesExperiencia[i], " num jugadores:", poblacion[i])
                if(poblacion[i] != 0):
                        nivelesExperiencia[i] /= poblacion[i]

        grafico_barras_apiladas(niveles,nivelesExperiencia,labels,'Niveles','Media de Muertes','Media de muertes por experiencia en videojuegos')

#ARREGLAR!!
def graficoDifWallJumpVsTiempoWallJump():
        dificultad = ['Muy fácil', 'Fácil', 'Difícil', 'Muy Difícil']
    
        mediaJ = np.zeros(len(playersArray))
        mediaTiempoWJ = np.zeros(4)

        zonas=0
        for p in playersArray:
                for i in range(len(p.tiempoZonasEspecialesPorNivel)):
                        for j in range(len(p.tiempoZonasEspecialesPorNivel[i])):
                                mediaJ[p]+= p.tiempoZonasEspecialesPorNivel[i][j]
                                zonas+=1
                mediaJ[p]/= zonas
                zonas=0
                
                dif=p.walljump
                if dif == '1':
                        mediaTiempoWJ[0]+=mediaJ[p]
                elif dif == '2':
                        mediaTiempoWJ[1]+=mediaJ[p]
                elif dif == '3':
                        mediaTiempoWJ[2]+=mediaJ[p]
                elif dif == '4':
                        mediaTiempoWJ[3]+=mediaJ[p]

        poblacion = numJugadoresPorDifWJ()

        for i in range(len(poblacion)):
                #print("muertes:", nivelesExperiencia[i], " num jugadores:", poblacion[i])
                if(poblacion[i] != 0):
                        mediaTiempoWJ[i] /= poblacion[i]

        
        grafico_barras(dificultad,mediaTiempoWJ,'Dificultad de WJ percivida por Tiempo medio en WJs', '', 'Tiempo Medio')


grafico_rangosEdad()
#no va
#graficoDifWallJumpVsTiempoWallJump()
#grafico_paths()
graficoMuertesPorNivel()
graficoMuertesVSEdad()
graficoMediaMuertesVSExp()
graficoMediaMuertesVsDificultad()

#graficoMuertesVSDificultad()
#graficoMuertesVSExpGeneral()
#graficoMuertesVSExpPlataformas()

